In [ ]:
import torch
import transformers

# model_name = "gpt2"
model_name = "microsoft/phi-2"
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, temperature=0.2, trust_remote_code=True, torch_dtype=torch.bfloat16)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
prompt = "The meaning of life is"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=50, do_sample=True)
print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
import outlines
from human_eval.data import HUMAN_EVAL, stream_jsonl
problems = list(stream_jsonl(HUMAN_EVAL))

In [ ]:
len(problems)

In [ ]:
problems[:2]

In [ ]:
@outlines.prompt
def few_shots(instructions, examples, question):
    """{{ instructions }}

    {% for example in examples %}
    Question:
    ```
    {{ example.prompt }}
    ```
    Answer:
    ```
    {{ example.canonical_solution }}
    ```
    {% endfor %}

    Question:
    ```
    {{ question }}
    ```
    Answer:
    ```
    """

instructions = "Please answer the following question following the examples. Generate valid python code by indenting 4 spaces always."
examples = problems[:2]
# [
#     {"question": "2+2=?", "answer":4},
#     {"question": "3+3=?", "answer":6}
# ]
question = problems[3]['prompt']

prompt = few_shots(instructions, examples, question)
print(prompt)

In [ ]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_new_tokens=80, do_sample=True)
print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
output_str = tokenizer.decode(output[0, input_ids.shape[-1]:], skip_special_tokens=True)
output_clean_str = output_str.split("```")[0]
print(output_clean_str)

In [ ]:
output_str = tokenizer.decode(output[0, input_ids.shape[-1]:], skip_special_tokens=True)
output_clean_str = output_str.split("```")[0]
print(output_clean_str)

In [ ]:
print(problems[3]['canonical_solution'])

In [ ]:
deep_tokenizer = transformers.AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Python-hf", trust_remote_code=True)

In [ ]:
prompt = "        def"
deep_tokenizer(prompt, return_tensors="pt")

In [ ]:
deep_tokenizer.decode([308])